In [1]:
import os

os.environ['OPENAI_API_KEY'] = "sk-pzmVDXe1-rxgtV34fEob57Owp9b7Z7_k-jYKmFaguDT3BlbkFJXR835WsPGSV7P_sjJXS2QGOJBIxgW5wSzwqExklvAA"

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from typing import List


In [9]:
def get_all_file_paths(folder) -> List[str]:
    file_paths = []
    for root, _, files in os.walk(folder):
        for file in files:
            suffix = os.path.splitext(file)[1]
            if suffix != ".md":
                continue
            if "説明用" in file or "sample" in file:
                continue
            file_paths.append(os.path.join(root, file))
    return file_paths

In [15]:
def create_retriever_for_multiple_files(folder_path):
    file_paths = get_all_file_paths(folder_path)
    # print(len(file_paths))
    if len(file_paths) == 0:
        raise ValueError(f"フォルダ『{folder_path}」の下にルールが見つからないです")
    splits = []
    for file_path in file_paths:
        loader = TextLoader(file_path, encoding="utf-8-sig")
        docs = loader.load()
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        
        splits.extend(text_splitter.split_documents(docs))
    vectorstore = Chroma.from_documents(documents=splits, 
                                        embedding=OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY")))
    retriever = vectorstore.as_retriever(k=4)
    return retriever

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
question = "納品物作成1を紹介してください"
llm = ChatOpenAI(model="gpt-4o", api_key= os.getenv("OPENAI_API_KEY"))
folder_path = "/Users/formaideveloper/LLM_excel/application_v4_test/rule"
retriever = create_retriever_for_multiple_files(folder_path)
# Prompt
prompt = hub.pull("rlm/rag-prompt")
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

/Users/formaideveloper/miniconda3/envs/LLM_env/lib/python3.10/site-packages/langsmith/client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
question_2 = "納品物作成2-2詳細に紹介してください"
ans = rag_chain.invoke(question_2)
print(ans)

納品物作成2-2は、`shikyuukoujo_csv`の中で特定の`スタッフコード`に一致する行を対象に、データを処理して出力します。`shikyuukoujo_csv`の列を`sikyuukoujo_design_csv`の見出しに一致させ、振込先情報を`staff_kyuuyo_csv`を基に設定します。最終的に、組み替えたデータをデータフレームとして出力します。


In [18]:
question_3 = "全体はいくつのタスクがありますか,それぞれ教えて"
ans = rag_chain.invoke(question_3)
print(ans)

タスクは1つだけで、「納品物作成1-2」という名前のタスクがあります。


In [33]:
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

/Users/formaideveloper/miniconda3/envs/LLM_env/lib/python3.10/site-packages/langsmith/client.py:351: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]
